<a href="https://colab.research.google.com/github/huijunam/CJ_AI_hackathon/blob/eunhy/5G_v_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install ipyplot
!pip install transformers
!pip install torch>=1.8.1
!pip install keybert
!pip install konlpy
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'


  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-_xl1g5km/kobert-tokenizer_560dd618cab244a4adfafdde8ac41161
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-_xl1g5km/kobert-tokenizer_560dd618cab244a4adfafdde8ac41161
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [2]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import seaborn as sns
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import datetime
import time
import pandas as pd
import re

In [3]:
from google.colab import drive
import os
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
os.chdir('/content/drive/MyDrive/CJ')

In [5]:
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)
device

device(type='cuda')

In [6]:
!git clone https://github.com/huijunam/CJ_AI_hackathon.git

fatal: destination path 'CJ_AI_hackathon' already exists and is not an empty directory.


In [7]:
target_classes ={
    '공포': 0,
    '놀람': 1,
    '분노': 2,
    '슬픔': 3,
    '중립': 4,
    '행복': 5,
    '혐오': 6
  }

In [8]:
reverse =dict(map(reversed, target_classes.items()))

In [9]:
# 하이퍼 파라미터 설정
max_length = 100
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
random_seed = 42

In [10]:
model_name = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels =len(target_classes)).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/CJ/KcElectra10000_weights.pth'))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [11]:
def classify_emotion(text):
    # 텍스트를 토큰화하고 모델에 입력으로 전달
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)

    # 감정 분류 결과 가져오기
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).cpu().numpy()[0]

    # 예측된 감정 레이블 출력
    #emotion = target_classes[predicted_class]
    return predicted_class

In [12]:
from keybert import KeyBERT

In [13]:
kw_model = KeyBERT()

In [14]:
from konlpy.tag import Okt
okt = Okt()

In [15]:
s = pd.read_excel("/content/drive/MyDrive/CJ/stopwords.xlsx")
stop_words = set(list(s['stop_words']))

In [43]:
def predict_and_keword(predict_sentence):
    inputs = tokenizer(predict_sentence, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)

    # 감정 분류 결과 가져오기
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).cpu().numpy()[0]
    emotion = reverse.get(predicted_class)
    print(emotion)

    # 키워드 추출: 명사, 동사, 형용사 추출하기
    predict_sentence = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', '', predict_sentence)

    words = predict_sentence.split()
    word_count = len(words)
    number = math.ceil(word_count/2)

    keywords = kw_model.extract_keywords(predict_sentence, keyphrase_ngram_range=(1, 2),use_maxsum = True, top_n=number) #use_maxsum = True
    keyword = [x for x,y in keywords]
    keyword = ' '.join(keyword)

    key = [x for (x, y) in okt.pos(keyword, stem=True) if y in ['Noun',"Verb"]]
    keyv1 = [word for word in set(key) if not word in stop_words]
    print(keyv1)

    if emotion != '중립':
        keyv1.append(emotion)

    return keyv1

In [17]:
import json
import random
from urllib import parse, request
import re
from IPython.display import Image
import ipyplot

url = "http://api.giphy.com/v1/gifs/search"


        WARNING! Google Colab Environment detected!
        You might encounter issues while running in Google Colab environment.
        If images are not displaying properly please try setting `force_b64` param to `True`.
        


In [18]:
import math

In [47]:
sentence = input()
array = predict_and_keword(sentence)
print(array)
data_list = []
for item in array:
    params = parse.urlencode({
        "q": item,
        "api_key": "P9SzTTpADkMNu3XjLkzRyMxVGm3MyKDQ",
        "limit": "20"
        })

    try:
        with request.urlopen("".join((url, "?", params))) as response:
            data = json.loads(response.read())

        get_nums = set()

        while len(get_nums) < (len(data['data']))/2:
            get_nums.add(random.randint(0,len(data['data'])+1))

        for i in get_nums:
            #print(data['data'][i]['images']['downsized']['url'])
            data_list.append(data['data'][i]['images']['downsized']['url'])

    except:
        pass

get_nums = set()
while len(get_nums) != 10:
    try:
        get_nums.add(random.randint(0,len(data_list)-1))
    except:
        break

data_gif = []
for i in get_nums:
    data_gif.append(data_list[i])


for idx, value in enumerate(data_gif):
    data_gif[idx] = re.sub('media+[0-9]', 'i', value)

ipyplot.plot_images(data_gif,img_width=200,show_url=False)

그래서 난 제로칼로리 콜라가 좋음.
행복
['콜라', '제로', '칼로리']
['콜라', '제로', '칼로리', '행복']
